In [ ]:
import pymssql
import json
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import KFold, GridSearchCV
from datetime import datetime
from sklearn import metrics
from sklearn.externals import joblib
import datetime
from sklearn.metrics import mean_squared_error
from math import sqrt
import _mssql
from PIL import ImageTk, Image
from tkinter import *
from tkinter import ttk
import time
import threading
from tkinter import messagebox

In [ ]:
def dbaccessProc():
    
    timer = 0
    timerExecuteFlag = 0
    now  = 0
    indexData = 0
    printFlag = 0
    indexData = 0

    # 서버 로그인 정보 json 파일 불러오기
    json_data=open('C:/Users/dap/login.json').read()
    login = json.loads(json_data)

    if printFlag == 1 :
        print("DSRS CONNECT")
        print("DSRS GatheringData_IF READ")

    cnxn = pymssql.connect(server = login['server'], user = login['user'], password = login['password'],database = login['database'])
    cursor = cnxn.cursor()

    cmd = "select * from GatheringData_IF"
    cursor.execute(cmd)
    in_row = cursor.fetchone()                    
    
    while type(in_row) == 'NoneType':
        cursor.execute(cmd)
        in_row = cursor.fetchone() 
    
    data = pd.read_sql(cmd, cnxn) 

    datetime = pd.DatetimeIndex(data['TimeStamp'])
    data['month'] = datetime.month
    data['hour'] = datetime.hour
    data['dayofweek'] = datetime.dayofweek

    list_of_names = ['KeyValue', 'TimeStamp', 'Temperature', 'RelativeHumidity',
           'WindSpeed', 'SolarIrradiation', 'AtmospherePressure',
           'WindDirection', 'DustCurrent', 'VisibilityDistance',
           'YardVibration', 'SurfaceTemperature', 'PM10Yard',
           'PM10AirKorea', 'month', 'hour', 'dayofweek']
    data.columns = list_of_names

    inputdata = data.drop(['KeyValue', 'TimeStamp','PM10Yard'], axis=1)

    filename_1 = 'C:/Users/dap/model_1.sav'
    filename_2 = 'C:/Users/dap/model_2.sav'
    filename_3 = 'C:/Users/dap/model_3.sav'
    model_1 = joblib.load(filename_1)
    model_2 = joblib.load(filename_2)
    model_3 = joblib.load(filename_3)
    
    def predict_1(inputdata):
        predictions = model_1.predict(inputdata)
        predicted = pd.DataFrame(predictions, columns = ['predicted'])
        return predictions

    result_1 = predict_1(inputdata)
    
    def predict_2(inputdata):
        predictions = model_2.predict(inputdata)
        predicted = pd.DataFrame(predictions, columns = ['predicted'])
        return predictions

    result_2 = predict_2(inputdata)

    def predict_3(inputdata):
        predictions = model_3.predict(inputdata)
        predicted = pd.DataFrame(predictions, columns = ['predicted'])
        return predictions

    result_3 = predict_3(inputdata)
    
    present = [in_row[12]]
    
    MSE = mean_squared_error(present, result_1)
    RMSE_1 = sqrt(MSE)

    MSE = mean_squared_error(present, result_2)
    RMSE_2 = sqrt(MSE)

    MSE = mean_squared_error(present, result_3)
    RMSE_3 = sqrt(MSE)
    
    cmd = "select * from ModelData_IF"
    cursor.execute(cmd)
    out_row = cursor.fetchone()

    index_value = in_row[1]

    if printFlag == 1 :
        print ('ModelData_IF WRITE = ',index_value)


    cmd = "update ModelData_IF set TimeStamp = %s"

    cursor.execute(cmd, index_value)
    cnxn.commit()
    
    modela_pm = float("{:.3f}".format(result_1[0]))

    if printFlag == 1 :
        print ('ModelData_IF WRITE = ',modela_pm)

    cmd = "update ModelData_IF set ModelA = %s"

    cursor.execute(cmd, modela_pm)
    cnxn.commit() 

    modelb_pm = float("{:.3f}".format(result_2[0]))

    if printFlag == 1 :
        print ('ModelData_IF WRITE = ',modelb_pm)

    cmd = "update ModelData_IF set ModelB = %s"

    cursor.execute(cmd, modelb_pm)
    cnxn.commit() 

    modelc_pm = float("{:.3f}".format(result_3[0]))

    if printFlag == 1 :
        print ('ModelData_IF WRITE = ',modelc_pm)

    cmd = "update ModelData_IF set ModelC = %s"

    cursor.execute(cmd, modelc_pm)
    cnxn.commit() 
    
    modela_rmse = float("{:.3f}".format(RMSE_1))

    if printFlag == 1 :
        print ('ModelData_IF WRITE = ', RMSE_1)

    cmd = "update ModelData_IF set ModelA_rmse = %s"

    cursor.execute(cmd, modela_rmse)
    cnxn.commit() 

    modelb_rmse = float("{:.3f}".format(RMSE_2))

    if printFlag == 1 :
        print ('ModelData_IF WRITE = ', RMSE_2)

    cmd = "update ModelData_IF set ModelB_rmse = %s"

    cursor.execute(cmd, modelb_rmse)
    cnxn.commit() 
    
    modelc_rmse = float("{:.3f}".format(RMSE_3))

    if printFlag == 1 :
        print ('ModelData_IF WRITE = ', RMSE_3)

    cmd = "update ModelData_IF set ModelC_rmse = %s"

    cursor.execute(cmd, modelc_rmse)
    cnxn.commit() 
    
    cmd = "select * from ModelData_IF"
    cursor.execute(cmd)
    out_row = cursor.fetchone()
    
    cnxn.close()

    if printFlag == 1 :
        print("PSDB CLOSE")
    
    timer = threading.Timer(60, dbaccessProc)

    if printFlag == 1 :
        print('dbaccessProc Func Execute')

    timer.start()

    #global txt_i_index

    textDisplayStr = in_row[1]
    txt_i_index.delete(0, END)
    txt_i_index.insert(END, textDisplayStr)
    
    textDisplayStr = in_row[12]
    textDisplayStr = float("{:.3f}".format(textDisplayStr))
    txt_i_time.delete(0, END)
    txt_i_time.insert(END, textDisplayStr)

    textDisplayStr = out_row[0]

    txt_o_index.delete(0, END)

    txt_o_index.insert(END, textDisplayStr)

    textDisplayStr = out_row[1]

    txt_o_time.delete(0, END)

    txt_o_time.insert(END, textDisplayStr)

    textDisplayStr = out_row[2]

    txt_o_result_1.delete(0, END)

    txt_o_result_1.insert(END, textDisplayStr)
    
    textDisplayStr = out_row[3]

    txt_o_result_2.delete(0, END)

    txt_o_result_2.insert(END, textDisplayStr)
    
    textDisplayStr = out_row[4]

    txt_o_result_3.delete(0, END)

    txt_o_result_3.insert(END, textDisplayStr)
    
    textDisplayStr = out_row[5]

    txt_o_result_4.delete(0, END)

    txt_o_result_4.insert(END, textDisplayStr)
    
    textDisplayStr = out_row[6]

    txt_o_result_5.delete(0, END)

    txt_o_result_5.insert(END, textDisplayStr)


In [ ]:
def fun_a():
    
    global timer
    
    timer = threading.Timer(60, fun_a)
    timer.start()
    timerExecuteFlag = 1


root = Tk()
root.title("In&Out Program")
root.minsize(500,650)
root.maxsize(500,650)
frame = Canvas(root, height=650, width=500)
image1 = PhotoImage(file='C:/Users/dap/back.png')
frame.create_image(542,0 , anchor = NE, image=image1)
frame.pack(side=TOP)

def on_closing():
    global timerExecuteFlag
    if messagebox.askokcancel("Quit", "사용을 종료하시겠습니까?"):

        if timerExecuteFlag >= 1:
            strdis = 'timerExecuteFlag = {0}'.format(timerExecuteFlag)
            timer.cancel()
            timerExecuteFlag = 0

        root.destroy()

def okClick():
    global timerExecuteFlag

    timerExecuteFlag = 1
    dbaccessProc()

def cancelClick():
    
    global timerExecuteFlag

    if timerExecuteFlag >= 1:
        timer.cancel()
        timerExecuteFlag = 0


lbl_i_index = Label(frame, fg='White', bg="#212933", font="Bahnschrift 14", text="Input TIME")
lbl_i_index.place(x=80, y=220)
lbl_i_time = Label(frame, fg='White', bg="#212933",font="Bahnschrift 14",text="PM10")
lbl_i_time.place(x=80, y=250)
txt_i_index = Entry(frame, width = 25,fg='White', bg="#212933",bd=0,font="Arial")
txt_i_index.place(x=200, y=225)
txt_i_time = Entry(frame, width = 25, fg='White', bg="#212933",bd=0,font="Arial")
txt_i_time.place(x=200, y=255)

#OUTPUT VALUE DISPLAY

lbl_o_index = Label(frame, fg='White', bg="#212933",font="Bahnschrift 14",text="Output TIME")
lbl_o_index.place(x=80, y=305)

lbl_o_time = Label(frame, fg='White', bg="#212933",font="Bahnschrift 14",text="MODEL_A")
lbl_o_time.place(x=80, y=335)

lbl_o_result = Label(frame, fg='White', bg="#212933",font="Bahnschrift 14",text="MODEL_B")
lbl_o_result.place(x=80, y=365)

lbl_o_result = Label(frame, fg='White', bg="#212933",font="Bahnschrift 14",text="MODEL_C")
lbl_o_result.place(x=80, y=395)

lbl_o_result = Label(frame, fg='White', bg="#212933",font="Bahnschrift 14",text="A_RMSE")
lbl_o_result.place(x=80, y=425)

lbl_o_result = Label(frame, fg='White', bg="#212933",font="Bahnschrift 14",text="B_RMSE")
lbl_o_result.place(x=80, y=455)

lbl_o_result = Label(frame, fg='White', bg="#212933",font="Bahnschrift 14",text="C_RMSE")
lbl_o_result.place(x=80, y=485)

txt_o_index = Entry(frame, width = 25, fg='White', bg="#212933",bd=0,font="Arial")
txt_o_index.place(x=200, y=310)

txt_o_time = Entry(frame, width = 25, fg='White', bg="#212933",bd=0,font="Arial")
txt_o_time.place(x=200, y=340)

txt_o_result_1 = Entry(frame, width = 25, fg='White', bg="#212933",bd=0,font="Arial")
txt_o_result_1.place(x=200, y=370)

txt_o_result_2 = Entry(frame, width = 25, fg='White', bg="#212933",bd=0,font="Arial")
txt_o_result_2.place(x=200, y=400)

txt_o_result_3 = Entry(frame, width = 25, fg='White', bg="#212933",bd=0,font="Arial")
txt_o_result_3.place(x=200, y=430)

txt_o_result_4 = Entry(frame, width = 25, fg='White', bg="#212933",bd=0,font="Arial")
txt_o_result_4.place(x=200, y=460)

txt_o_result_5 = Entry(frame, width = 25, fg='White', bg="#212933",bd=0,font="Arial")
txt_o_result_5.place(x=200, y=490)


 
# 버튼 클릭 이벤트와 핸들러 정의
btn = Button(frame, text="START",  fg='White', bg="#212933" ,font="Bahnschrift 12",command=okClick) 
btn.place(x=305, y=580)
# 버튼 클릭 이벤트와 핸들러 정의
btn1 = Button(frame, text="STOP",  fg='White', bg="#212933",font="Bahnschrift 12",command=cancelClick) 
btn1.place(x=370, y=580)

# 버튼 클릭 이벤트와 핸들러 정의
btn2 = Button(frame, text="EXIT",  fg='White', bg="#212933",font="Bahnschrift 12",command=on_closing) 
btn2.place(x=425, y=580)

root.protocol("WM_DELETE_WINDOW", on_closing) 
root.mainloop()